<a href="https://colab.research.google.com/github/aleef-19/Personal_project/blob/main/Saans_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#List of Required dependencies to install
!pip install -q -U transformers evaluate accelerate datasets fsspec==2025.3.2
!pip install -q rouge_score bert_score
!pip install -q huggingface_hub torch tqdm evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 105.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [1]:
#HF Login
from huggingface_hub import login
login()

In [ ]:
#Dataset loading
#We will be using the aci-bench dataset available in hugging face, as it has been pre-processed by the community\
#It has the same number number of conversations as the aci-bench dataset with both the soap style and narrative style note available in one file\
#Split into train,validation,test-(177,10,20) of total 207 conversations all in one file same as ACI Bench which was split over many tasks\
#for the purpose of cross-benchmarking over different tasks

#https://huggingface.co/datasets/ClinicianFOCUS/ACI-Bench-Refined (link to the dataset)

import pandas as pd

df = pd.read_parquet("hf://datasets/ClinicianFOCUS/ACI-Bench-Refined/data/validation-00000-of-00001.parquet")

conv    = df["dialogue"]
soap    = df["augmented note"]
note    = df["note"]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
#PROMPT for SOAP note generation
SOAP_PROMPT = """
**Role:** You are an AI assistant specialized in generating clinical SOAP notes.

**Task:** Generate a concise, accurate, and clinically relevant SOAP note based **STRICTLY AND SOLELY** on the provided doctor-patient interaction transcript.

**CRITICAL INSTRUCTIONS:**

1.  **Strict Transcript Adherence:** Generate the SOAP note using **ONLY** information **explicitly stated** within the provided transcript.
2.  **NO Assumptions or External Knowledge:** **DO NOT** infer information, add details not mentioned (even if clinically likely), make assumptions, or use external medical knowledge. Adherence to the transcript is paramount.
3.  **Standard SOAP Structure:** Organize the output clearly into the following sections using **EXACTLY** these headings:
    *   **S – Subjective**
    *   **O – Objective**
    *   **A – Assessment**
    *   **P – Plan**
4.  **NO Extraneous Text:** The output must contain **ONLY** the four section headings (S, O, A, P) and the corresponding content derived *directly* from the transcript. **DO NOT** include introductory sentences (e.g., "Here is the SOAP note:"), concluding remarks, disclaimers, notes about the generation process, metadata, or *any* other text before, between, or after the S/O/A/P sections.

**Formatting:**

*   Use clear headings for each SOAP section (as listed above).
*   Be concise but ensure all relevant details *from the transcript* are included under the correct heading.
*   Use standard medical abbreviations only if they are unambiguous and directly supported by the transcript's terminology.

**Input:** You will receive a doctor-patient transcript.

**Output:** Generate **ONLY** the structured SOAP note (S/O/A/P sections and content) based on the critical instructions above.

SOAP NOTE:
    """

In [ ]:
#PROMPT for Clinical note generation
NOTE_PROMPT = """
**Role:** You are an AI assistant specialized in generating structured clinical notes.

**Task:** Generate a concise, accurate, and clinically relevant structured clinical note based **STRICTLY AND SOLELY** on the provided doctor-patient interaction transcript.

**CRITICAL INSTRUCTIONS:**

1.  **Strict Transcript Adherence:** Use **ONLY** information **explicitly stated** within the provided transcript.

2.  **NO Assumptions or External Knowledge:** **DO NOT** infer information, add details not mentioned (even if clinically likely), make assumptions, or use external medical knowledge. Adherence to the transcript is paramount..

3.  **Standard Note Structure:** Organize the note using **EXACTLY** these headings:

    *   **CHIEF COMPLAINT**

    *   **HISTORY OF PRESENT ILLNESS**

    *   **REVIEW OF SYSTEMS**

    *   *PAST MEDICAL HISTORY**

    *   **MEDICATIONS**

    *   **ALLERGIES**

    *   **FAMILY HISTORY**

    *   **SOCIAL HISTORY**

    *   **PHYSICAL EXAM**

    *   **ASSESSMENT**

    *   **PLAN**

4.  **NO Extraneous Text:** The output must contain **ONLY** the section headings above and their corresponding content drawn *directly* from the transcript.**DO NOT** introductory sentences (e.g., "Here is the clinical note:"), concluding remarks, disclaimers, notes about the generation process, metadata, or *any* other text before, between, or after the sections.

**Formatting:**

  *   List each heading exactly as shown (all caps).
  *   Place the relevant content immediately beneath each heading.
  *   Be concise while ensuring every pertinent detail from the transcript appears under the correct heading.
  *   Use standard medical abbreviations only if they are unambiguous and explicitly present in the transcript.

**Input:** You will receive a doctor-patient transcript.

**Output:** Generate **ONLY** the structured clinical narrative note based on the critical instructions above.
NARRATIVE NOTE:
    """

# List of Models Selected for Evaluation on the ACI-Bench Dataset

We plan to benchmark the following models on the ACI-Bench clinical dataset:

- **LLaMa 3.2 3B**  
- **Qwen-3 4B**  
- **Clinician-Note-2.0a** (Qwen-3 4B fine-tuned on ACI-Bench training and validation datasets)  

We are specifically using instruction-tuned language models or models that support `chat_template` formatting.  
Models with a `chat_template` typically perform significantly better compared to base LLMs that lack this structure.  
Additionally, fine-tuning LLMs with an existing `chat_template` is more efficient, as these models already have a standardized conversational format, which provides a strong base for task-specific generation.

This approach ensures that our benchmark focuses on models optimized for clinical dialogue generation and evaluation.





In [ ]:
#List of models from Hugging Face
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

#Define models and their checkpoint names

models = {
    "LLaMa-3.2-3B":{
        "checkpoint": "meta-llama/Llama-3.2-3B-Instruct",
        "parameters":  "3 Billion"
    },
    "Qwen3-4B": {
        "checkpoint": "Qwen/Qwen3-4B",
        "parameters": "4 Billion"
    },
    "Clinician-Note-2.0a": {
        "checkpoint": "ClinicianFOCUS/Clinician-Note-2.0a",
        "parameters": "4 Billion"
    }
}

In [ ]:
#Generates the notes based on the required format and model specifications
import torch
from tqdm import tqdm

def generate_notes(dialogues, tokenizer, model, prompt, *,
                   max_new_tokens: int = 2048, do_sample: bool = False):
    """
    Return pure SOAP notes, one-per-dialogue.
    Processes dialogues one-by-one to avoid exceeding context window limits.
    """
    notes = []
    model.eval()

    # Ensure pad_token_id is set
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id

    with torch.inference_mode():
        for d in tqdm(dialogues):
            # Build chat-prompt
            text = tokenizer.apply_chat_template(
                [
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": d}
                ],
                tokenize=False
            )

            # Tokenize
            inputs = tokenizer(
                text,
                return_tensors="pt",
                truncation=True,
                padding=True
            ).to(model.device)

            # Compute input length properly
            input_len = inputs["input_ids"].shape[1]

            # Generate
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=do_sample
            )

            # Slice off the prompt and decode
            gen_ids = outputs[0][input_len:]
            note = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()
            notes.append(note)

    return notes

In [ ]:
#Sliding window for BERT as they only support up to 512 tokens at max
from typing import List, Tuple
from transformers import AutoTokenizer
from statistics import mean
import evaluate

def sliding_window_chunks(
    text: str,
    tokenizer: AutoTokenizer,
    max_tokens: int = 512,
    stride: int = 256
) -> List[str]:
    """
    Break a long string into overlapping windows of ≤ max_tokens (sub-word pieces).

    Returns a *list of substrings*.  No padding—each chunk is an actual slice of `text`.
    """
    tokens = tokenizer(text, add_special_tokens=False)["input_ids"]
    windows = []
    start = 0
    while start < len(tokens):
        end = min(start + max_tokens, len(tokens))
        window_ids = tokens[start:end]
        windows.append(tokenizer.decode(window_ids, skip_special_tokens=True))
        if end == len(tokens):
            break
        start += stride
    return windows


In [ ]:
#Calculates ROUGE scores
def rouge_scores_mean(
    cands: List[str],
    refs:  List[str],
    rouge_types: Tuple[str, ...] = ("rouge1", "rouge2", "rougeL")
) -> dict:
    """
    Median ROUGE-F1 for each rouge_type, expressed as a percentage.
    """
    rouge = evaluate.load("rouge")
    raw = rouge.compute(
        predictions=cands,
        references=refs,
        rouge_types=rouge_types,
        use_stemmer=True,
        use_aggregator=False   # ← one score per example
    )
    # raw looks like {'rouge1': [f1_pair1, f1_pair2, ...], 'rouge2': [...], ...}
    return {k: round(mean(v) * 100, 2) for k, v in raw.items()}


In [ ]:
# Loading tokenizers of models to be used for evaluating

import evaluate
from transformers import AutoTokenizer

bert_scorer  = evaluate.load("bertscore")
bert_tok     = AutoTokenizer.from_pretrained("microsoft/deberta-base-mnli")


In [ ]:
# bertscore_sliding

from statistics import mean

def bertscore_sliding(cands, refs,
                      max_tokens=512, stride=256):
    f1s = []
    for cand, ref in zip(cands, refs):
        c_chunks = sliding_window_chunks(cand,  bert_tok, max_tokens, stride)
        r_chunks = sliding_window_chunks(ref,   bert_tok, max_tokens, stride)
        for i in range(max(len(c_chunks), len(r_chunks))):
            c = c_chunks[i] if i < len(c_chunks) else c_chunks[-1]
            r = r_chunks[i] if i < len(r_chunks) else r_chunks[-1]
            score = bert_scorer.compute(
                predictions=[c], references=[r],
                model_type="microsoft/deberta-base-mnli"
            )["f1"][0]
            f1s.append(score)
    return mean(f1s)

In [ ]:
#Running models on the conversations and calculating their scores on metrics for SOAP notes
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd

soap_results = []

for name, cfg in models.items():    #Loads the models
    print(f"🔹 Running {name}")
    tokenizer = AutoTokenizer.from_pretrained(cfg["checkpoint"], use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(
        cfg["checkpoint"],
        device_map="auto",
        torch_dtype=torch.float16
    )

    cands = generate_notes(conv.tolist(), tokenizer, model, SOAP_PROMPT)

    rouge = rouge_scores_mean(cands, soap.tolist())   #ROUGE scores
    bert_f1 = bertscore_sliding(cands, soap.tolist())   #BERT scores

    soap_results.append({
        "model": name,
        **rouge,
        "bertscore_f1": round(bert_f1*100, 2)
    })

    # free GPU
    del model, tokenizer
    gc.collect()
    torch.cuda.empty_cache()

🔹 Running LLaMa-3.2-3B


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

100%|██████████| 10/10 [03:33<00:00, 21.39s/it]


,model,rouge1,rouge2,rougeL,bertscore_f1
0,LLaMa-3.2-3B,45.72,18.71,26.28,66.97
1,Qwen3-4B,43.33,18.59,28.05,66.96
2,Clinician-Note-2.0a,44.14,19.40,29.52,67.21


In [ ]:
#Running models on the conversations and calculating their scores on metrics for Clinical notes
note_results = []

for name, cfg in models.items():    #Loads the models
    print(f"🔹 Running {name}")
    tokenizer = AutoTokenizer.from_pretrained(cfg["checkpoint"], use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(
        cfg["checkpoint"],
        device_map="auto",
        torch_dtype=torch.float16
    )

    cands = generate_notes(conv.tolist(), tokenizer, model, NOTE_PROMPT)

    rouge = rouge_scores_mean(cands, note.tolist())   #Rouge scores

    note_results.append({
        "model": name,
        **rouge
    })

    # free GPU
    del model, tokenizer
    gc.collect()
    torch.cuda.empty_cache()

🔹 Running LLaMa-3.2-3B


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
 10%|█         | 1/10 [00:27<04:08, 27.56s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
 20%|██        | 2/10 [00:52<03:26, 25.86s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
 30%|███       | 3/10 [01:14<02:49, 24.21s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_to

🔹 Running Qwen3-4B


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 10/10 [04:21<00:00, 26.19s/it]


🔹 Running Clinician-Note-2.0a


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 10/10 [04:25<00:00, 26.56s/it]


,model,rouge1,rouge2,rougeL,bertscore_f1
0,LLaMa-3.2-3B,51.86,20.90,27.99,67.21
1,Qwen3-4B,46.16,18.05,26.30,67.21
2,Clinician-Note-2.0a,47.28,19.34,27.49,67.21


In [ ]:
print(pd.DataFrame(soap_results))  #DataFrame with models and their metric wise scores

                 model  rouge1  rouge2  rougeL  bertscore_f1
0         LLaMa-3.2-3B   45.72   18.71   26.28         66.97
1             Qwen3-4B   43.33   18.59   28.05         66.96
2  Clinician-Note-2.0a   44.14   19.40   29.52         67.21
                 model  rouge1  rouge2  rougeL  bertscore_f1
0         LLaMa-3.2-3B   51.86   20.90   27.99         67.21
1             Qwen3-4B   46.16   18.05   26.30         67.21
2  Clinician-Note-2.0a   47.28   19.34   27.49         67.21


In [ ]:
print(pd.DataFrame(note_results))

                 model  rouge1  rouge2  rougeL  bertscore_f1
0         LLaMa-3.2-3B   51.86   20.90   27.99         67.21
1             Qwen3-4B   46.16   18.05   26.30         67.21
2  Clinician-Note-2.0a   47.28   19.34   27.49         67.21


##Fine-Tuning Details of Models in the ACI-Bench Evaluation

[ACI-BENCH abstract](https://arxiv.org/abs/2306.02022)
Data Based on ACI-Bench Research Paper

**BART + SAMSum (Division-FT)**  
First continued-pre-training on the *SAMSum* dialogue-summary corpus, then fine-tuned for **10 epochs on ACI-BENCH**, one note-division at a time.

**BART (Division-FT)**  
The vanilla BART-Large checkpoint, fine-tuned on the **ACI-BENCH training split** with exactly the same division-by-division recipe and hyper-parameters as above.

**LED (Division-FT)**  
Longformer-Encoder-Decoder; loaded from the public checkpoint and fine-tuned on **ACI-BENCH divisions** (longer 2k-token inputs) using the same training schedule as BART.

---

## ROUGE Scores
(Narrative Style Notes)(ACI)

| Model                            | ROUGE-1 | ROUGE-2 | ROUGE-L |
|----------------------------------|---------|---------|---------|
| BART + SAMSum (Division-FT)      | 53.46   | 25.08   | 48.62   |
| GPT-4                            | 51.76   | 22.58   | 45.97   |
| BART (Division-FT)               | 51.56   | 24.06   | 45.92   |
| Text-davinci-003                 | 47.07   | 22.08   | 43.11   |
| ChatGPT (gpt-3.5-turbo)          | 47.44   | 19.01   | 42.47   |
| Retrieval (train UMLS)           | 43.87   | 17.55   | 40.47   |
| LED (Division-FT)                | 34.15   | 8.01    | 29.80   |


(Custom)

| Model                            | ROUGE-1 | ROUGE-2 | ROUGE-L |
|----------------------------------|---------|---------|---------|
| LLaMa-3.2-3B                     | 51.86   | 21.74   | 27.99   |
| Qwen3-4B                         | 46.16   | 18.05   | 26.30   |
| Clinician-Note-2.0a              | 47.28   | 19.34   | 27.49   |


---

## ROUGE and BERT Scores
(SOAP Format Notes)

| Model                            | R-1     | R-2     | R-L     | BERT  |
|----------------------------------|---------|---------|---------|-------|
| BART + SAMSum (Division-FT)      | 43.4    | 24.8    | 31.7    | 70.1  |
| GPT-4                            | 41.8    | 20.3    | 31.9    | 66.9  |
| BART (Division-FT)               | 42.4    | 24.8    | 31.6    | 69.5  |
| Text-davinci-003                 | 41.9    | 21.9    | 34.4    | 67.6  |
| ChatGPT (gpt-3.5-turbo)          | 38.3    | 17.9    | 28.9    | 66.7  |
| Retrieval (train UMLS)           | 40.0    | 21.4    | 32.0    | 70.8  |
| LED (Division-FT)                | 25.1    | 6.8     | 14.1    | 54.0  |


(Custom)

| Model                            | R-1     | R-2     | R-L     | BERT  |
|----------------------------------|---------|---------|---------|-------|
| LLaMa-3.2-3B                     | 45.22   | 18.58   | 25.98   | 66.92 |
| Qwen3-4B                         | 45.33   | 18.59   | 28.05   | 66.96 |
| Clinician-Note-2.0a              | 44.14   | 19.40   | 29.52   | 67.21 |


**Reference: ACI-BENCH Benchmark Results**  
*(These numbers are from the original ACI-BENCH research paper and were not re-implemented in this notebook.)*

---

### Key Findings from Both Experiments

1. **Prompt Engineering Alone**  
   - **Setup:** Zero-shot narrative and SOAP templates enhanced with explicit section headers (e.g., “HPI,” “Assessment and Plan”).  
   - **Result:** Increased ROUGE-1 score over retrieval-based models and almost achieves the same performance as fine-tuned models, sometimes outperforming them when writing in structured formats.  
   - **Inference:** Structuring the prompt to mirror clinical note sections gives the model clear guidance, boosting accuracy without any model updates. Properly engineered prompts can nearly match fine-tuned performance.

2. **Retrieval-Augmented Generation (RAG) Alone**  
   - **Setup:** Prepending the top-k most relevant dialogue snippets (via simple sentence embeddings) to a standard few-shot prompt.  
   - **Result:** Increased ROUGE-1 score versus zero-shot prompting. Retrieval-based models can catch up to or even surpass fine-tuned models when writing in a structured way (SOAP).  
   - **Inference:** Grounding the model in patient-specific context enhances faithfulness and reduces omissions, even without prompt restructuring, as it already has examples that are finely structured.

3. **Fine-Tuned Models vs. Prompt/RAG**  
   - **Observation:** Fully fine-tuned models (e.g., BART+FT, GPT, LLaMA, DaVinci, Qwen) achieve higher **ROUGE-L** scores due to learning exact consecutive phrasing patterns.  
   - **RAG Performance:** In SOAP-style notes, RAG-based outputs often outperform these fine-tuned LLMs by supplying precise section content.  
   - **Narrative Trade-off:** RAG approaches may lag on open-ended narrative notes, where the free generative capacity of fully fine-tuned models yields richer phrasing and coherence.

---

### Suggestions to Further Improve Accuracy

- **Combine Context-Rich Prompts with RAG**  
  Fusing section-aware templates with retrieved snippets leverages both structural guidance and precise content grounding—likely pushing ROUGE-1 scores significantly higher.

- **One-Shot Demonstrations to GPT-3.5**  
  Present a single annotated example mapping “Dialogue → [Section Header] → Note Text.” This concise demonstration can teach formatting and content alignment in one go. Combined with a contextually rich prompt, accuracy would improve further.

- **Chain-of-Thought Prompting**  
  Instruct the model to “think aloud” for each section (“First, identify subjective complaints; then extract objective findings…”). Breaking the task into substeps improves completeness and factuality. This would enable the model to generate a much better Assessment and Plan section.

- **Domain-Specific Conditioning**  
  Prepend instructions like “Use UMLS concepts for key findings” or include a mini-glossary. Anchoring terminology reduces generic paraphrasing and ensures medical accuracy.

- **Output Reranking / Post-Processing**  
  Generate multiple candidates and rerank them using a lightweight ROUGE-L or rule-based factuality check. Simple reranking often yields a 1–2 point lift.

---

### Multi-Model Pipeline for Extraction + Summarization

1. **Extraction Model:**  
   - Ingest raw EHR data or scanned images (via OCR) to extract structured findings, vitals, lab results, and other key elements.

2. **Summarization Model:**  
   - Consume the extracted data and, using chain-of-thought prompting, generate a SOAP-style note—reasoning sequentially through Subjective, Objective, Assessment, and Plan.

**Benefits:**  
- **Reduced Hallucinations:** The summarizer works from verified extractions rather than free-form text.  
- **Higher Accuracy:** Chain-of-thought ensures methodical processing of each section.  
- **Format Consistency:** Fine-tuning the summarizer on SOAP-structured notes teaches it to maintain layout conventions.

---

By integrating these strategies—especially the synergy of prompt engineering and retrieval, alongside a split extraction/summarization architecture—you can surpass many fine-tuned baselines without heavy training, achieving high-fidelity clinical note generation in a lightweight, scalable pipeline.  

# Ways to Improve Model Accuracy

<details>
<summary><strong>1. Chain-of-Thought (CoT) Prompting</strong></summary>

### Description:
Chain-of-Thought prompting forces the model to reason step-by-step:
- Step 1: Identify patient-reported symptoms (Subjective)
- Step 2: Extract clinician observations, vitals, and test results (Objective)
- Step 3: Infer the diagnosis (Assessment)
- Step 4: Propose the treatment plan (Plan)

### Advantages:
- Enforces logical, sequential reasoning.
- Reduces hallucination by guiding the model through fact validation.
- Improves factual consistency across SOAP sections.

### Drawbacks:
- Longer prompts → Higher API latency and token usage.
- May exceed token limits for long conversations.(If context window small)

### API-Specific Notes:
- Chain-of-Thought significantly increases token consumption → Higher cost.
- Suitable for API calls but requires careful cost management.

</details>

---

<details>
<summary><strong>2. Division-Wise Fine-Tuning (LoRA Heads per SOAP Section)</strong></summary>

### Description:
Use separate LoRA adapters fine-tuned for each SOAP section:
- Head 1: Subjective
- Head 2: Objective
- Head 3: Assessment
- Head 4: Plan

### Advantages:
- Reduces input size per call → Lower latency and API token usage.
- Section-specific specialization improves clinical accuracy and writing style.
- More structured SOAP note generation.

### Drawbacks:
- Requires orchestration: 4 sequential API calls per note (one per head).
- Multi-step pipeline increases total processing time.
- May lose cross-section coherence since each head is isolated.

### API-Specific Notes:
- Multi-call orchestration required.
- Can increase cumulative latency and total token usage.
- Parallelizing API calls would speed up the process.

</details>

---

<details>
<summary><strong>3. Retrieval-Augmented Generation (RAG)</strong></summary>

### Description:
Retrieve top-k similar clinical encounters to prepend to the API input.

### Advantages:
- Improves factual grounding → Reduces hallucination.
- Adapts to rare or edge cases by using prompt-specific retrievals.
- Enhances writing structure by providing real-world examples.

### Drawbacks:
- Requires fast retrieval system before API call.
- Long retrieval examples could push prompt length close to API token limits.

### API-Specific Notes:
- Retrieval preprocessing adds minor overhead but manageable.
- Prompt length increases → Requires balancing retrieval size.
- Retrieval database must be well-curated for clinical accuracy.

</details>

---


<details>
<summary><strong>4. Post-Processing with Regex and RxNorm</strong></summary>

### Description:
- Apply regex to standardize vitals, lab values, units, and dates.
- Use RxNorm to normalize medication names.

### Advantages:
- Improves EHR compatibility and machine readability.
- Fixes formatting errors that the LLM may introduce.
- Fast and computationally inexpensive.

### Drawbacks:
- Cannot fix conceptual errors like incorrect treatment plans.
- Regex patterns may miss complex phrasing variations.

### API-Specific Notes:
- Post-processing is done after API call → No additional API cost.
- Lightweight, scalable solution → No impact on API token usage.

</details>

---





# Better Evaluation Metrics

<details>
<summary><strong>1. Traditional Metrics: ROUGE, BLEURT, BERTScore</strong></summary>

### What They Capture:
- **ROUGE**: Measures surface-level n-gram, word-sequence and word-pair overlaps to assess recall of reference content.  
- **BLEURT**: Uses a pretrained transformer fine-tuned on human judgments to score semantic similarity and penalize factual drift.  
- **BERTScore**: Computes cosine similarity between contextual token embeddings, capturing soft overlaps and paraphrases.

### What They Miss:
- **Clinical Correctness**: Cannot verify if dosages, lab values or diagnoses are accurately represented.  
- **Semantic Hallucinations**: Fails to penalize plausible-sounding but incorrect medical details.  
- **Nuanced Paraphrasing**: May underrate valid clinical restatements that differ lexically from the gold standard.

### API-Specific Notes:
- **Local Evaluation** → Zero API calls, minimal latency.  
- **Fast but Shallow** → Suitable for nightly regression tests but not for high-stakes clinical validation.  
- **Integration** → Easily added to existing CI pipelines, but should be supplemented for clinical domains.

</details>

---

<details>
<summary><strong>2. LLM-Based Scoring</strong></summary>

### What It Captures:
- **Faithfulness**: Checks if generated text aligns with source dialogue and key patient facts.  
- **Clinical Completeness**: Evaluates inclusion of all SOAP components (Subjective, Objective, Assessment, Plan).  
- **Fluency & Appropriateness**: Judges coherence, terminology usage, and adherence to clinical style guides.

### What It Misses:
- **Model Hallucination**: An LLM evaluator itself might introduce or overlook errors without transparent reasoning.  
- **Over- or Under-Scoring**: Subject to evaluator prompt framing; minor phrasing changes can impact scores unpredictably.

### API-Specific Notes:
- **Cost & Latency**: Each note requires a separate evaluation call, increasing token usage and round-trip time.  
- **Prompt Engineering**: Detailed, chain-of-thought style prompts improve reliability but consume more tokens.  
- **Parallelization**: Batch multiple notes in a single request (if supported) or parallel async calls to optimize throughput.

</details>

---

<details>
<summary><strong>3. Chain-of-Thought Evaluation</strong></summary>

### What It Captures:
- **Step-by-Step Reasoning**:  
  1. **Extract Key Facts**: Identify vital signs, symptoms, and clinical impressions from the transcript.  
  2. **Cross-Check Presence**: Verify that each fact appears correctly in the note’s Subjective and Objective sections.  
  3. **Score Fidelity**: Assign a quantitative score based on coverage, accuracy, and terminology precision.

### What It Misses:
- **Complex Phrasing**: Subtle distinctions or implied meaning can still be misinterpreted by the evaluator.  
- **Increased Overhead**: Multi-step evaluation multiplies token consumption and evaluation time.

### API-Specific Notes:
- **Longer Prompts** → Higher token usage per evaluation, impacting cost.  
- **Enhanced Reliability** → More transparent scoring rationale, reducing “black-box” errors.  
- **Prompt Tuning** → Requires iterative refinement to balance depth of reasoning against cost constraints.

</details>

---

<details>
<summary><strong>4. MEDCON-Based Fact Checking</strong></summary>

### What It Captures:
- **Symbolic Fact Matching**: Uses medical concept ontologies to map and compare entities (e.g., ICD-10 codes, SNOMED terms) between source and note.  
- **Direct Consistency**: Flags missing or contradictory lab values, medication names, and dosages.

### What It Misses:
- **Paraphrased Details**: Cannot detect when a clinician rephrases information (e.g., “elevated blood sugar” vs. “hyperglycemia”).  
- **Contextual Nuance**: Lacks understanding of implied clinical reasoning or differential diagnoses.

### API-Specific Notes:
- **Local Execution** → No API costs, ultra-fast for large batches.  
- **Scalability** → Ideal for high-volume pipelines, but should be combined with semantic evaluators.  
- **Maintenance** → Requires periodic updates of concept dictionaries and ontologies.

</details>

---

<details>
<summary><strong>Combined Evaluation Strategy</strong></summary>

### Recommended Workflow:
1. **Baseline**: Run ROUGE, BLEURT, BERTScore for quick regression checks.  
2. **Symbolic Verification**: Apply MEDCON for factual consistency on key clinical entities.  
3. **Deep Judgment**: Use LLM-based scoring with chain-of-thought prompts to assess nuanced clinical fidelity.  
4. **Post-Processing**: Standardize formatting, terminology, and ensure compliance with documentation guidelines.

### API-Specific Notes:
- **Token Budget**: Monitor cumulative token usage across draft generation and multiple evaluation calls.  
- **Batching & Async**: Group multiple notes into fewer API requests or leverage asynchronous libraries to hide latency.  
- **Cost Optimization**: Prioritize cheaper local checks first; reserve expensive LLM evaluations for borderline or high-priority cases.

</details>

---



## References

1. **arXiv Paper**  
   *Title:* [Paper: arXiv:2306.02022](https://arxiv.org/pdf/2306.02022)

2. **ACI-Bench GitHub Repository**  
   *Source:* [https://github.com/wyim/aci-bench](https://github.com/wyim/aci-bench)

3. **ACI-Bench Refined Dataset (Hugging Face)**  
   *Dataset:* [https://huggingface.co/datasets/ClinicianFOCUS/ACI-Bench-Refined](https://huggingface.co/datasets/ClinicianFOCUS/ACI-Bench-Refined)

4. **Clinician-Note-2.0a (Hugging Face)**

    [https://huggingface.co/ClinicianFOCUS/Clinician-Note-2.0a]